# Problem set 9: Statistics, feature selection, and feature importance

## Summary

Examine the differences between British and American fiction in the class-curated literary corpus. Apply statistical measures and calculate feature importance in a simple classifier.

## Details

You will work with a corpus of 131 volumes of fiction by British and American authors. These volumes are taken from the class corpus, so you'll need to download a copy of the texts from [Google Drive](https://drive.google.com/drive/folders/1lbeZiBAVCzjCWojCK8mfmELa-Q8FMNUm?usp=sharing) or from GitHub and save them somewhere on your machine.

You have three tasks for this problem set, all of which depend on comparing British-authored to American-authored books:

1. Calculate the mean frequency per 100,000 words, as well as the upper and lower bounds of a 95% confidence interval, for the terms `['color', 'honor', 'center', 'fish', 'person']` in each national subcorpus
    1. Perform this calculation analyticaly, that is, using the observed sample means and standard deviations.
    1. Calculate the same quantities via bootstrap, using 1,000 or more iterations.
    1. In both cases, print your results in a tabular format.
2. Perform a *t*-test to compare the mean frequency of each of these terms between British and American texts. Report the test statistic and *p*-value for each comparison. Note which means are significantly different at the *p*<0.05 level.
3. Perform a logistic regression classification of each volume as British or American. 
    1. Your final features should be the 25 most informative (as measured by the mutual information criterion) token unigrams.
    1. Report your 10-fold cross-validated F1 score before and after restricting your input features to the 25 most-informative token types.
    1. Calculate the *importance* of the 25 top features for classification as measured by permutation importance.
    
* See code stubs below for step-by-step guidance. 
* Consult, too, the lecture notes on explainability and on statistics.
* You'll likely also need to consult the scikit-learn documentation along the way.

## Imports and setup

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np

metadata_file = 'amer_brit.csv'
corpus_dir = os.path.join('..', '..', 'data', 'classcorpus')
terms = ['color', 'honor', 'center', 'fish', 'person']

## Read metadata (5 points)

Read the cleaned, minimal corpus metadata from disk (note the variable `metadata_file` defined in the previous cell). I'd suggest using Pandas, but you're welcome to use whatever method you prefer.

Note that the format of the metadata file is:
```
filename,country,wordcount
```

In [2]:
corpus = pd.read_csv(metadata_file)
# Read the corpus metadata

In [3]:
# Print the metadata for one volume
corpus.head(1)
# corpus.head(10)

,filename,country,wc
0,Little_Women_Alcott.txt,us,185902


## Count words and normalize (5 points)

* Count the target words (indicated in the problem statement) in each volume. 
* Then, **normalize the count of each word type per 100,000 words** in each volume.
*  I'd suggest using a `CountVectorizer` object, but again, you may approach this task however you like. 
* Make sure you lowercase the input tokens.
* Use the word counts supplied in the metadata file for length normalization.

In [4]:
# Count and normalize the target terms in each volume as indicated
#done during section
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(input= 'filename', vocabulary= terms)
filenames = [os.path.join('..', '..', 'data', 'classcorpus', f) for f in list(corpus['filename'])]
count = vectorizer.fit_transform(filenames)

In [5]:
wc = corpus['wc'].to_numpy() #done during section- with teammates
normalized = count/wc[:,None] * 100000

print(normalized)

[[ 11.29627438  15.599617     0.           3.76542479  12.37211004]
 [  0.           0.           0.           9.24257128  13.86385692]
 [  0.           0.           4.47007286  26.82043717   8.94014572]
 [  0.           0.           0.           0.          25.95413904]
 [  0.           0.           0.           0.          38.42551454]
 [  0.           1.68693804   0.           0.84346902  58.19936234]
 [  0.           0.           0.           4.11373659  41.13736589]
 [  0.           0.           0.           0.          21.31207141]
 [  0.           0.           0.           1.27037362  42.55751617]
 [ 35.66969859   0.          10.19134245   0.          12.73917807]
 [  8.30344095   7.74987822   0.           1.10712546  10.51769186]
 [  0.           0.           0.           1.34772706  32.3454494 ]
 [  5.42947117   0.           5.42947117  43.43576936   5.42947117]
 [ 12.73236567   0.           6.36618284   0.          12.73236567]
 [  0.           0.           0.           0.   

In [6]:
# Print the normalized term frequencies you just calculated for any three documents
wc = corpus['wc'].to_numpy()
wc = wc[:,None] 
# print(wc)
t = (count.toarray())
corpus['color'] = 0
corpus['honor'] = 0
corpus['center'] = 0
corpus['fish'] = 0
corpus['person'] = 0 
#print(t)
pd.set_option('mode.chained_assignment', None) #the error is actually not an error it is a warning (warning due to poor code structure, but still right)
#can comment out to see error- it will have no effect on the code 
for i in range(len(corpus)): 
    corpus['color'][i]= t[i][0]
    corpus['honor'][i] = t[i][1]
    corpus['center'][i] = t[i][2]
    corpus['fish'][i] = t[i][3]
    corpus['person'][i] = t[i][4]

In [7]:
corpus['color_norm'] = 0.0
corpus['honor_norm'] = 0.0
corpus['center_norm'] = 0.0
corpus['fish_norm'] = 0.0
corpus['person_norm'] = 0.0 
pd.set_option('mode.chained_assignment', None) #same as above
for i in range(len(corpus)): 
    corpus['color_norm'][i] = (corpus['color'][i]/wc[i]) * 100000
    corpus['honor_norm'][i] = (corpus['honor'][i]/wc[i]) * 100000
    corpus['center_norm'][i] = (corpus['center'][i]/wc[i]) * 100000
    corpus['fish_norm'][i] = (corpus['fish'][i]/wc[i]) * 100000
    corpus['person_norm'][i] = (corpus['person'][i]/wc[i]) * 100000
corpus.head()

,filename,country,wc,color,honor,center,fish,person,color_norm,honor_norm,center_norm,fish_norm,person_norm
0,Little_Women_Alcott.txt,us,185902,21,29,0,7,23,11.296274,15.599617,0.000000,3.765425,12.372110
1,The_Great_God_Pan.txt,gb,21639,0,0,0,2,3,0.000000,0.000000,0.000000,9.242571,13.863857
2,The_Lost_Kafoozalum.txt,gb,22371,0,0,1,6,2,0.000000,0.000000,4.470073,26.820437,8.940146
3,NORTHANGER_ABBEY.txt,us,77059,0,0,0,0,20,0.000000,0.000000,0.000000,0.000000,25.954139
4,persuasion.txt,gb,83278,0,0,0,0,32,0.000000,0.000000,0.000000,0.000000,38.425515


### Note: 
Both of the values found by each of the two methods I presented above are the same, but I felt that although the first method was much easier to do, the second method provides a visual aid. 

## Calculate analytic means and 95% confidence intervals (15 points)

* For each of the five indicated terms, calculate and display the mean and 95% confidence interval within each national group.
*  I suggest using the `tconfint_mean()` method from the `DescrStatsW()` function provided by the `statsmodels` library. See lecture notes for an example of working code.
* Format your output (roughly) as follows:

```
Confidence intervals for: gb
     term	    low	    mean	    high
   color	  x.xxxx	  x.xxxx	  x.xxxx
   [and so on ...]
```

In this part of the problem, calculate your means and CIs analytically, using the observed statistics of each sample, rather than by bootstrapping.

In [8]:
corpus.describe()

,wc,color,honor,center,fish,person,color_norm,honor_norm,center_norm,fish_norm,person_norm
count,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000
mean,79314.694656,3.648855,2.625954,2.061069,4.335878,20.908397,6.898563,3.721598,3.577896,6.362920,23.747075
std,54486.012043,6.783377,5.720860,5.080514,15.180762,28.769495,13.935690,7.608938,7.732523,15.343624,18.704961
min,2721.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47876.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,8.621587
50%,60099.000000,0.000000,0.000000,0.000000,1.000000,13.000000,0.000000,0.000000,0.000000,1.548119,21.295569
75%,92321.500000,5.000000,2.000000,1.000000,3.000000,25.500000,10.486021,3.428566,4.022918,4.040888,32.613140
max,331988.000000,47.000000,31.000000,34.000000,163.000000,218.000000,104.877940,56.511911,41.652782,112.426923,92.931077


In [9]:
# Calculate and display analytic means and CIs
import statsmodels.stats.api as sms

In [10]:
gb = corpus.loc[corpus['country']=='gb'] 
us = corpus.loc[corpus['country']=='us'] 
for i in terms: 
    gb_i = sms.DescrStatsW(gb[i+'_norm']).tconfint_mean()
    gb_mean = gb[i+'_norm'].mean()
    print(i , 'gb')
    print('Confidence Interval:', gb_i , 'Mean:', gb_mean)
print('')
for i in terms: 
    us_i = sms.DescrStatsW(us[i+'_norm']).tconfint_mean()
    us_mean = us[i+'_norm'].mean()
    print(i , 'us')
    print('Confidence Interval:', us_i , 'Mean:', us_mean)
    

color gb
Confidence Interval: (0.05734754735849712, 1.204246452081989) Mean: 0.6307969997202431
honor gb
Confidence Interval: (-0.2501225410326629, 1.102709793545379) Mean: 0.42629362625635797
center gb
Confidence Interval: (-0.007464714372635295, 1.1855245885963273) Mean: 0.5890299371118458
fish gb
Confidence Interval: (1.717683649779449, 6.703498791980557) Mean: 4.210591220880003
person gb
Confidence Interval: (22.378088325893255, 31.80637495662347) Mean: 27.092231641258362

color us
Confidence Interval: (8.639749919684554, 17.131614347797083) Mean: 12.885682133740813
honor us
Confidence Interval: (4.601434723833549, 9.137269677288321) Mean: 6.869352200560936
center us
Confidence Interval: (4.050608922728463, 8.815255118870965) Mean: 6.432932020799715
fish us
Confidence Interval: (3.7917039117609717, 13.046048516897141) Mean: 8.418876214329059
person us
Confidence Interval: (16.13440211575759, 24.969001015479595) Mean: 20.551701565618597


## Calculate bootstrapped means and 95% confidence intervals (15 points)

* Calculate the same quantities as above, but this time by bootrap resampling of your data. 
* Use a minimum of 1,000 trials for each case. 
* Format your results as in the previous question.

In [11]:
# Bootstrap calculations
#from lecture
import random
import numpy as np

gb = corpus.loc[corpus['country']=='gb'] 
us = corpus.loc[corpus['country']=='us'] 

# bootstrapped_means_gb = []
trials = 1000
for i in terms:
    bootstrapped_means_gb = []
    k = gb[i].count()
    for x in range(trials):
        sample = gb[i].sample(n=k, replace=True) #create sample- code given by TA
        bootstrapped_means_gb.append(np.mean(sample))
    result_gb = sorted(bootstrapped_means_gb)
    low_gb = result_gb[int(trials*0.025)] 
    high_gb = result_gb[int(trials*0.975)] 
    print('gb:',i ,'\n', "Mean:", result_gb[int(trials/2)],'\n' ,"95% CI:", '(',low_gb, ',' ,high_gb, ')')
print('')
for i in terms:
    bootstrapped_means_us = []
    k = us[i].count()
    for x in range(trials):
        sample = us[i].sample(n=k, replace=True) #create sample 
        bootstrapped_means_us.append(np.mean(sample))
    result_us = sorted(bootstrapped_means_us)
    low_us = result_us[int(trials*0.025)] 
    high_us = result_us[int(trials*0.975)] 
    print('us:',i ,'\n', "Mean:", result_us[int(trials/2)],'\n' ,"95% CI:", '(',low_us, ',' ,high_us, ')')

gb: color 
 Mean: 0.21875 
 95% CI: ( 0.0625 , 0.421875 )
gb: honor 
 Mean: 0.140625 
 95% CI: ( 0.015625 , 0.375 )
gb: center 
 Mean: 0.234375 
 95% CI: ( 0.046875 , 0.65625 )
gb: fish 
 Mean: 2.46875 
 95% CI: ( 1.359375 , 4.0625 )
gb: person 
 Mean: 27.6875 
 95% CI: ( 20.125 , 38.265625 )

us: color 
 Mean: 6.850746268656716 
 95% CI: ( 5.134328358208955 , 9.014925373134329 )
us: honor 
 Mean: 4.925373134328358 
 95% CI: ( 3.462686567164179 , 6.746268656716418 )
us: center 
 Mean: 3.7313432835820897 
 95% CI: ( 2.283582089552239 , 5.462686567164179 )
us: fish 
 Mean: 5.880597014925373 
 95% CI: ( 2.5223880597014925 , 11.761194029850746 )
us: person 
 Mean: 14.119402985074627 
 95% CI: ( 10.582089552238806 , 18.776119402985074 )


## *t*-tests (20 points)

* Perform a *t*-test comparing the mean frequency of each of the indicated terms in the British and American subsets of the corpus.
    * You will perform 5 total tests, comparing, for example, the mean frequency of `color` in British texts to the mean frequency of `color` in American texts. Do not cross-compare words (that is, don't compare the frequency of `color` to that of `honor`, etc.).
* Note that the *t*-test takes as input two lists of values. These values are the normalized counts for the feature in question in each volume of a subcorpus. There should thus be one list per subcorpus for each feature. You can produce these lists on the fly as you iterate over your feature data.
* Display the test statistic and *p*-value for each comparison. 
    * Format your output for easy readability (do not just print the raw `ttest_ind` object).
* Note which differences are significant at the *p*<0.05 level. 

In [12]:
# Perform t-tests
# code from lecture
from scipy.stats import ttest_ind
terms_norm = ['color_norm', 'honor_norm', 'center_norm', 'fish_norm', 'person_norm']
for col in terms_norm:
    print(f'\n{col}:')
    result = ttest_ind(
        gb[col], 
        us[col],
        equal_var=False
    )
    print('t-statistic:', result[0])
    print('p-value:    ', result[1])


color_norm:
t-statistic: -5.710855578615899
p-value:     2.66232957951557e-07

honor_norm:
t-statistic: -5.435939141902402
p-value:     6.095694670555516e-07

center_norm:
t-statistic: -4.751214779484977
p-value:     9.648991593476994e-06

fish_norm:
t-statistic: -1.59890142906163
p-value:     0.11296901234604427

person_norm:
t-statistic: 2.0223115383604093
p-value:     0.04522772184736863


## Feature selection (25 points)

* Vectorize the corpus as indicated below (freebie)
* Standard-scale the resulting feature matrix
* Produce a one-dimensional label vector, y, indicating the national origin of each volume in the corpus
    * Use `1` to indicate American, `0` for British
* Calculate the 10-fold cross-validated classification accuracy and F1 score using a logistic regression classifier on the full input matrix
* From the full matrix, select the 25 most-informative features
    * Use sklearn's `SelectKBest` function with the  `mutual_info_classif` scoring function to produce a feature matrix that contains just these 25 most-informative features
    * Print a list of the names (token labels; for example, 'color') of these 25 features

In [13]:
# Vectorize (freebie)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

def pre_proc(x):
    '''
    Takes a unicode string.
    Lowercases, strips accents, and removes some escapes.
    Returns a standardized version of the string.
    '''
    import unicodedata
    return unicodedata.normalize('NFKD', x.replace("_", " ").lower().strip())

# Set up vectorizer
vectorizer = TfidfVectorizer(
    input='filename',
    encoding='utf-8',
    preprocessor=pre_proc,
    min_df=11, # Note this
    max_df=0.8, # This, too
    binary=False,
    norm='l2',
    max_features=5000,
    use_idf=True # And this
)

# Perform vectorization
X = vectorizer.fit_transform(filenames) # <-- MODIFY TO USE THE LIST OF FILES ON YOUR MACHINE

# Get the dimensions of the doc-term matrix
print("Matrix shape:", X.shape)

Matrix shape: (131, 5000)


In [14]:
# Standard-scale your feature matrix
scaler = StandardScaler()
X_standard = scaler.fit_transform(X.toarray())

In [15]:
# Print the overall mean of your scaled features (use np.mean(X)).
# Should be very close to zero.
print(np.mean(X_standard))

-6.942707647121589e-18


In [16]:
# Produce a one-dimensional vector of true labels for classification
# 1='us', 0='gb'
y = []
for i in range(len(corpus)): 
    if corpus['country'][i] == 'us': 
        y.append(1)
    else:
        y.append(0)
y = np.array(y)

In [17]:
# Using your label vector, display the number of US texts in the corpus
display(np.sum(y))

67

In [18]:
# Freebie function to summarize and display classifier scores
def compare_scores(scores_dict, color=True):
    '''
    Takes a dictionary of cross_validate scores.
    Returns a color-coded Pandas dataframe that summarizes those scores.
    '''
    import pandas as pd
    df = pd.DataFrame(scores_dict).T.applymap(np.mean)
    if color:
        df = df.style.background_gradient(cmap='RdYlGn')
    return df

In [19]:
# Cross-validate the logistic regression classifier on full input data
# Consult PS 6 for useful code- code below from pset
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
classifiers = {
    'Logit':LogisticRegression()
}
scores = {} # Store cross-validation results in a dictionary
for classifier in classifiers: 
    scores[classifier] = cross_validate( # perform cross-validation
        classifiers[classifier], # classifier object
        X_standard, # feature matrix
        y, # actual labels
        cv=10, #number of folds
        scoring=['accuracy', 'f1', 'f1_macro', 'f1_micro'] # scoring methods
    )

In [20]:
# Display your cross-validation results
# Use the compare_scores function defined above
compare_scores(scores)

,fit_time,score_time,test_accuracy,test_f1,test_f1_macro,test_f1_micro
Logit,0.073678,0.002364,0.847802,0.843337,0.846109,0.847802


In [21]:
# Select the 25 most-informative features as specified above 
#  and produce a new feature matrix containing only those features
from sklearn.feature_selection import SelectKBest, mutual_info_classif
best_25= SelectKBest(score_func = mutual_info_classif, k = 25)
new_feat = best_25.fit_transform(X_standard,y)

In [22]:
# Print the shape of your new feature matrix
print(new_feat.shape)

(131, 25)


In [23]:
# Get the names of the features retained in the new feature matrix
# Store these feature names in a list, then print the list

# Hint: use a combination of your original vectorizer's `.get_feature_names()` method 
#  and the `SelectKBest` object's `.get_support()` method

#code from: https://stackoverflow.com/questions/39839112/the-easiest-way-for-getting-feature-names-after-running-selectkbest-in-scikit-le

mask = best_25.get_support()
names = vectorizer.get_feature_names()
new = []

for bool, feature in zip(mask, names):
    if bool:
        new.append(feature)

print(new)

['afterwards', 'brief', 'color', 'colored', 'colour', 'coloured', 'daisy', 'duly', 'england', 'events', 'extraordinary', 'grey', 'hats', 'honor', 'honour', 'honoured', 'humour', 'london', 'neighbourhood', 'notes', 'recognise', 'thereby', 'toward', 'towards', 'upwards']


In [24]:
# Calculate and display the 10-fold cross-validated accuracy and F1 of the
#  logistic regression using the new, smaller feature matrix
new_scores = {} 
for classifier in classifiers: 
    new_scores[classifier] = cross_validate( # perform cross-validation
        classifiers[classifier], # classifier object
        new_feat, # feature matrix
        y, # actual labels
        cv=10, #number of folds
        scoring=['accuracy', 'f1', 'f1_macro', 'f1_micro'] # scoring methods
    )
compare_scores(new_scores)

,fit_time,score_time,test_accuracy,test_f1,test_f1_macro,test_f1_micro
Logit,0.010948,0.003978,0.924176,0.918022,0.922643,0.924176


In [25]:
c_scores = {}
all_feats = {
    'old': X_standard,
    'new': new_feat
}
for i in all_feats: 
    c_scores[i] = cross_validate(
        LogisticRegression(), 
        all_feats[i], 
        y, 
        cv=10, 
        scoring=['accuracy', 'f1', 'f1_macro', 'f1_micro'] # scoring methods
    )

compare_scores(c_scores)

,fit_time,score_time,test_accuracy,test_f1,test_f1_macro,test_f1_micro
old,0.097114,0.004100,0.847802,0.843337,0.846109,0.847802
new,0.008194,0.003508,0.924176,0.918022,0.922643,0.924176


## Identify the 5 most important features (15 points)

* Split the new matrix of most-informative features into train (75%) and test (25%) sets (use sklearn's `train_test_split`)
* Train a default logistic regression classifier on the training set
    * Print the trained model's score on the test set (use the trained classifier's `.score()` method)
* Use sklearn's `permutation_importance` function to calculate the importance of each input feature
* Print the feature importances from most to least important using the supplied function

In [26]:
# Split the selected feature matrix into train and test sets
# Then, train a logistic regression classifier on the train set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_feat, y, test_size=0.25, random_state=42)
model = LogisticRegression().fit(X_train, y_train)

In [27]:
# Print the score of the trained classifier on the test set
print(model.score(X_test,y_test))

0.9696969696969697


In [28]:
# Calculate feature importance via permutation
from sklearn.inspection import permutation_importance
p_feature = permutation_importance(model, new_feat, y)
print(p_feature)

{'importances_mean': array([ 0.01221374, -0.00152672,  0.03358779,  0.00763359,  0.00305344,
        0.01374046,  0.00152672,  0.01374046,  0.01526718,  0.0351145 ,
        0.00152672,  0.02442748,  0.01374046,  0.02137405,  0.01679389,
       -0.00152672,  0.        ,  0.00305344,  0.01068702,  0.00305344,
        0.        ,  0.01832061,  0.0610687 ,  0.01832061,  0.06717557]), 'importances_std': array([0.01142491, 0.00305344, 0.00778476, 0.00682769, 0.00778476,
       0.0101271 , 0.00305344, 0.01480208, 0.01365538, 0.0103547 ,
       0.00747936, 0.01313332, 0.00890222, 0.01221374, 0.01121904,
       0.00571245, 0.0048279 , 0.00610687, 0.0103547 , 0.01142491,
       0.        , 0.00778476, 0.01365538, 0.00778476, 0.01556953]), 'importances': array([[ 0.00763359,  0.02290076, -0.00763359,  0.02290076,  0.01526718],
       [ 0.        ,  0.        , -0.00763359,  0.        ,  0.        ],
       [ 0.03053435,  0.03053435,  0.03816794,  0.02290076,  0.04580153],
       [ 0.01526718,  0.

In [29]:
# Freebie function to print ranked list of feature importances
def print_importances(object_importance, feature_names):
    '''
    Takes a trained permutation_importance object and a list of feature names.
    Prints an ordered list of features by descending importance.
    '''
    for i in object_importance.importances_mean.argsort()[::-1]:

        print(f"{feature_names[i]:<8}"
            f"\t{object_importance.importances_mean[i]:.3f}"
            f" +/- {object_importance.importances_std[i]:.3f}")

In [30]:
# Print ranked list of features by permutation importance
print_importances(p_feature, new)

upwards 	0.067 +/- 0.016
toward  	0.061 +/- 0.014
events  	0.035 +/- 0.010
color   	0.034 +/- 0.008
grey    	0.024 +/- 0.013
honor   	0.021 +/- 0.012
towards 	0.018 +/- 0.008
thereby 	0.018 +/- 0.008
honour  	0.017 +/- 0.011
england 	0.015 +/- 0.014
coloured	0.014 +/- 0.010
duly    	0.014 +/- 0.015
hats    	0.014 +/- 0.009
afterwards	0.012 +/- 0.011
neighbourhood	0.011 +/- 0.010
colored 	0.008 +/- 0.007
london  	0.003 +/- 0.006
notes   	0.003 +/- 0.011
colour  	0.003 +/- 0.008
extraordinary	0.002 +/- 0.007
daisy   	0.002 +/- 0.003
humour  	0.000 +/- 0.005
recognise	0.000 +/- 0.000
honoured	-0.002 +/- 0.006
brief   	-0.002 +/- 0.003


# Top 5 
1. upwards
2. toward
3. events
4. color
5. grey